In [1]:
import os
import pandas as pd
import glob


In [2]:
# Specify the directory containing the CSV files
directory = r'data/'  # Use raw string to handle backslashes

# Read each CSV file into a DataFrame, add a new column, and store them in a list
dataframes = []
for dirpath, _, filenames in os.walk(directory):
    for file in filenames:
        if file.endswith('.csv'):
            file_path = os.path.join(dirpath, file)
            try:
                df = pd.read_csv(file_path)
                df['source_file'] = os.path.relpath(file_path, directory)  # Add a new column with the relative file path
                dataframes.append(df)
            except FileNotFoundError:
                print(f"File not found: {file_path}")
            except pd.errors.EmptyDataError:
                print(f"File is empty: {file_path}")
            except Exception as e:
                print(f"Error reading {file_path}: {e}")

# Optionally, concatenate all DataFrames into a single DataFrame
if dataframes:
    all_data = pd.concat(dataframes, ignore_index=True)
    # Now 'all_data' contains all the data from the CSV files with an additional column 'source_file'
    print(all_data)
else:
    print("No dataframes to concatenate.")

                date   time  temp                  source_file
0         2018-01-01  00:00  20.6    2018\2018\201801\ABRT.csv
1         2018-01-01  01:00  20.1    2018\2018\201801\ABRT.csv
2         2018-01-01  02:00  20.1    2018\2018\201801\ABRT.csv
3         2018-01-01  03:00  19.6    2018\2018\201801\ABRT.csv
4         2018-01-01  04:00  19.1    2018\2018\201801\ABRT.csv
...              ...    ...   ...                          ...
49780603  2023-12-31  19:00  28.6  2023\2023\202312\YOM012.csv
49780604  2023-12-31  20:00  26.4  2023\2023\202312\YOM012.csv
49780605  2023-12-31  21:00  25.1  2023\2023\202312\YOM012.csv
49780606  2023-12-31  22:00  24.2  2023\2023\202312\YOM012.csv
49780607  2023-12-31  23:00    24  2023\2023\202312\YOM012.csv

[49780608 rows x 4 columns]


In [ ]:
temp = pd.DataFrame(dataframes)

In [4]:
# Specify the directory containing the CSV files
directory = r'data/'  # Use raw string to handle backslashes
# Get all CSV files in the directory
csv_files = glob.glob(os.path.join(directory, "*.csv"))
print(csv_files)

# dataframes = []
# try:
#   # Read all CSV files into a list of DataFrames
#   dataframes = pd.read_csv(csv_files, low_memory=False)
# except pd.errors.EmptyDataError as e:
#   print(f"One or more files are empty: {e}")
# except Exception as e:
#   print(f"Error reading CSV files: {e}")

[]


In [ ]:
# Optionally, concatenate all DataFrames into a single DataFrame
all_data = pd.concat(dataframes, ignore_index=True)

# Now 'all_data' contains all the data from the CSV files with an additional column 'source_file'
print(all_data)